In [15]:
from pynq import Overlay
ol = Overlay('matrix.bit')

In [16]:
ol?
dma = ol.axi_dma_0

In [17]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [18]:
import numpy as np
import random 

In [19]:
np.random.seed(2)
data_A = np.random.randint(low=0, high=10, size=(100, 100))
data_B = np.random.randint(low=0, high=10, size=(100, 100))

In [20]:
data_A, data_B

(array([[8, 8, 6, ..., 4, 2, 9],
        [7, 1, 9, ..., 1, 5, 9],
        [4, 9, 8, ..., 5, 0, 1],
        ...,
        [8, 7, 2, ..., 0, 5, 1],
        [0, 1, 8, ..., 5, 2, 9],
        [3, 0, 1, ..., 4, 1, 3]]),
 array([[5, 7, 1, ..., 1, 8, 9],
        [3, 6, 5, ..., 0, 5, 9],
        [3, 0, 1, ..., 5, 2, 4],
        ...,
        [6, 4, 2, ..., 4, 1, 0],
        [3, 1, 2, ..., 0, 0, 7],
        [8, 0, 7, ..., 7, 8, 8]]))

In [21]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            for k in range(100):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [22]:
import time
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

6.686463832855225


In [23]:
from pynq import allocate

In [24]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [25]:
data_input

array([8, 8, 6, ..., 7, 8, 8])

In [26]:
input_buffer = allocate(20000, np.int32)
output_buffer = allocate((100,100), np.int32)

In [27]:
np.copyto(input_buffer, data_input)

In [28]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.0016713142395019531


In [29]:
output_buffer

PynqBuffer([[2167, 2183, 1892, ..., 1980, 2188, 2325],
            [2048, 2017, 1690, ..., 1952, 2090, 2098],
            [1782, 1871, 1706, ..., 1874, 1871, 1973],
            ...,
            [1976, 1789, 1614, ..., 1851, 2080, 2091],
            [2209, 2124, 2136, ..., 2338, 2367, 2291],
            [1980, 2071, 1893, ..., 2236, 2382, 1929]], dtype=int32)

In [30]:
diff = np.dot(data_A, data_B) - output_buffer

In [31]:
#rms error 
rms_error = np.sqrt(np.mean(diff**2))

In [32]:
diff

PynqBuffer([[0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]])

In [33]:
rms_error

PynqBuffer(0.)

In [34]:
%%time

np.dot(data_A, data_B)

CPU times: user 8.57 ms, sys: 26 µs, total: 8.6 ms
Wall time: 7.81 ms


array([[2167, 2183, 1892, ..., 1980, 2188, 2325],
       [2048, 2017, 1690, ..., 1952, 2090, 2098],
       [1782, 1871, 1706, ..., 1874, 1871, 1973],
       ...,
       [1976, 1789, 1614, ..., 1851, 2080, 2091],
       [2209, 2124, 2136, ..., 2338, 2367, 2291],
       [1980, 2071, 1893, ..., 2236, 2382, 1929]])